In [3]:
W = WeylGroup("C4", prefix="s")   # Ili tip kojeg ti želiš
[s1,s2,s3,s4] = W.simple_reflections()  # Ne zaboravi uštimati ovo ako promijeniš rank od W.

##################################################################################

n = rank(W) 

# Trivijalni i najduži element u W:
e = W(1)
w0 = W.long_element()

# Elemente od W dobiješ množeći proste refleksije, npr. w = s1*s2*s3*s1


##### Parabolic subgroups #####

# 'Sigma' sadrži redne brojeve prostih korijena iz Levija.
# E.g. za paraboličku p = o-x-o=<=o bilo bi Sigma=[1,3,4].

def minimal_rep(x,Sigma):
    '''The minimal representative of the class W_Sigma * x.'''   
    return x.coset_representative(Sigma, side='left')

def w0_(Sigma):
    '''The maximal element in the parabolic subgroup W_Sigma.'''
    return w0 * (minimal_rep(w0,Sigma)).inverse()

def W_(Sigma):
    '''Parabolic subgroup W_Sigma as a list.'''
    return W.bruhat_interval(e,w0_(Sigma))

def Minimal_reps(Sigma):
    '''Elements of W^p, i.e. minimal coset representatives.'''
    Min_reps = []
    Done = set()
    w0_Sigma = w0_(Sigma)
    for x in W:
        if x not in Done:
            Min_reps.append(minimal_rep(x,Sigma))
            Done=Done.union(W.bruhat_interval(minimal_rep(x,Sigma), w0_Sigma*minimal_rep(x,Sigma)))
    return Min_reps


def Minimal_reps_poset(Sigma):
    '''Often too big to see anything,
    but calling Minimal_reps_poset(Sigma).plot(figsize=15) might help.'''
    return (W.bruhat_poset()).subposet(Minimal_reps(Sigma))
# Strelice pokazuju u smjeru od Bruhat manjeg prema većem.
# Dakle, kao kod homomorfizama homogenih snopova, a suprotno od homomorphizama Vermaovih modula.


# Dodatno: 
# svi reducirani izrazi od w: w.reduced_words()
# Duljina od w: w.length()
# Za provjeriti vrijedi li je v<= w u Bruhatovom uređaju: v.bruhat_le(w)



##### Roots and weights ##### 

V =  W.domain()
Pi = V.simple_roots()
R_plus = V.positive_roots()
Lambda = V.fundamental_weights()
rho = V.rho()

def rho_(Sigma):
    '''The weight rho^p from [Baston-Eastwood, p. 40].'''
    return V(sum(V.fundamental_weights()[i] for i in range(1,rank(W)+1) if i not in Sigma))

# Za definirati vlasite težine koriste se epsilon-koordinate, npr: mu = V((1,0,-1,2))
# Za nul-težinu: V(0)
# Djelovanje od w \in W na težinu mu: w.action(mu)
# Nemoj koristiti 'lambda' kao varijablu, jer je to ključna riječ u Sage-u.

def dot_action(w,mu):
    return w.action(mu + rho) - rho

def Phi(w):
    '''The set Delta(w) from [Baston-Eastwood, p. 40].'''
    return [beta for beta in R_plus if (w.inverse()).action(beta) not in R_plus]

# Za provjeriti je li 'mu' dominantna integralna: mu.is_dominant_weight()
# Za provjeriti je li 'mu' dominantna: mu.is_dominant()
# Za provjeriti je li 'mu' dominantna za Levi od paraboličke definirane sa Sigma: mu.is_dominant(index_set=Sigma)

def regular_BGG(Sigma,mu):
    mu = V(mu)
    
    # Uvjet 
    if not mu.is_dominant_weight():
        raise Exception("Only dominant integral weights!")
    
    # Podesi ovdje veličinu outputa
    size = 15
    
    label = text("%s"%CartanType(W).marked_nodes([i for i in range(1,rank(W)+1) if i not in Sigma]).dynkin_diagram(),(0,0), axes=False)

    graphics = Minimal_reps_poset(Sigma).relabel(lambda w: str(dot_action(w,mu))+"\n"+ "".join(str(i) for i in w.reduced_word())  ).plot(
        figsize = size, vertex_shape="x", vertex_color="white",edge_color="gray"
    ) + label
    show(graphics)
#    graphics.save("%s_p=%s.png"%("".join(str(s) for s in CartanType(W)),"".join(str(s) for s in Sigma)))
    
# Ako želiš obično djelovanje umjesto dot-djelovanja, onda umjesto "(lambda w: dot_action(w,mu))"
# stavi "(lambda w: w.action(mu))", te gornji # Uvjet zamijeni sa
# if not (mu-rho).is_dominant_weight(): raise Exception("Only dominant integral regular weights!")    




#for Sigma in Subsets(range(1, rank(W)+1)):
#    regular_BGG(Sigma, V(0))
